In [1]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!mkdir -p `mecab-config --dicdir`
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd
!apt-get -q -y install swig
!pip install mecab-python3

Reading package lists...
Building dependency tree...
Reading state information...
sudo is already the newest version (1.8.21p2-3ubuntu1).
libmecab-dev is already the newest version (0.996-5).
mecab is already the newest version (0.996-5).
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
python-mecab is already the newest version (0.99.6-2).
curl is already the newest version (7.58.0-2ubuntu3.5).
file is already the newest version (1:5.32-2ubuntu0.1).
git is already the newest version (1:2.17.1-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-m

In [2]:
!pip install gensim

In [1]:
!ps aux
!kill -9 <pid>

USER         PID %CPU %MEM    VSZ   RSS TTY      STAT START   TIME COMMAND
root           1  0.0  0.0  39196  1508 ?        Ss   14:29   0:00 /bin/bash -e 
root           6  0.0  0.1 678212 21376 ?        Sl   14:29   0:00 node /tools/n
root          27  0.1  0.2 681068 32956 ?        Sl   14:29   0:01 /tools/node/b
root          51  0.1  0.3 187696 51948 ?        Sl   14:30   0:02 /usr/bin/pyth
root        2137  1.8  1.0 718264 135304 ?       Ssl  14:53   0:02 /usr/bin/pyth
root        2153  0.0  0.1  54376 14532 ?        S    14:53   0:00 /usr/bin/pyth
root        2161  0.0  0.0  59032  6380 ?        R    14:55   0:00 ps aux
/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `kill -9 <pid>'


create_model.py

In [3]:
import zipfile
import os.path
import urllib.request as req
import MeCab
from gensim import models
from gensim.models.doc2vec import TaggedDocument

#Mecabの初期化
mecab = MeCab.Tagger()
mecab.parse("")

#学習対象とする青空文庫の作品リスト --- (*1)
list = [
    {"auther":{
        "name":"宮澤 賢治",
        "url":"https://www.aozora.gr.jp/cards/000081/files/"}, 
     "books":[
        {"name":"銀河鉄道の夜","zipname":"43737_ruby_19028.zip"},
        {"name":"注文の多い料理店","zipname":"1927_ruby_17835.zip"},
        {"name":"セロ弾きのゴーシュ","zipname":"470_ruby_3987.zip"},
        {"name":"やまなし","zipname":"46605_ruby_29758.zip"},
        {"name":"どんぐりと山猫","zipname":"43752_ruby_17595.zip"},
    ]},
    {"auther":{
        "name":"芥川 竜之介",
        "url":"https://www.aozora.gr.jp/cards/000879/files/"}, 
     "books":[
        {"name":"羅生門","zipname":"127_ruby_150.zip"},
        {"name":"鼻","zipname":"42_ruby_154.zip"},
        {"name":"河童","zipname":"69_ruby_1321.zip"},
        {"name":"歯車","zipname":"42377_ruby_34744.zip"},
        {"name":"老年","zipname":"131_ruby_241.zip"},
    ]},
    {"auther":{
        "name":"太宰 治",
        "url":"https://www.aozora.gr.jp/cards/000035/files/"}, 
     "books":[
        {"name":"斜陽","zipname":"1565_ruby_8220.zip"},
        {"name":"走れメロス","zipname":"1567_ruby_4948.zip"},
        {"name":"津軽","zipname":"2282_ruby_1996.zip"},
        {"name":"お伽草紙","zipname":"307_ruby_3042.zip"},
        {"name":"人間失格","zipname":"301_ruby_5915.zip"},
    ]},
    {"auther":{
        "name":"夏目 漱石",
        "url":"https://www.aozora.gr.jp/cards/000148/files/"}, 
     "books":[
        {"name":"吾輩は猫である","zipname":"789_ruby_5639.zip"},
        {"name":"坊っちゃん","zipname":"752_ruby_2438.zip"},
        {"name":"草枕","zipname":"776_ruby_6020.zip"},
        {"name":"虞美人草","zipname":"761_ruby_1861.zip"},
        {"name":"三四郎","zipname":"794_ruby_4237.zip"},
    ]},
]

#作品リストを取得してループ処理に渡す --- (*2)
def book_list():
    for novelist in list:
        auther = novelist["auther"]
        for book in novelist["books"]:
            yield auther, book
    　　　 print("book_list ok")
        
#Zipファイルを開き、中の文書を取得する --- (*3)
def read_book(auther, book):
    zipname = book["zipname"]
    #Zipファイルが無ければ取得する
    if not os.path.exists(zipname):
        req.urlretrieve(auther["url"] + zipname, zipname)
    zipname = book["zipname"]
    #Zipファイルを開く
    with zipfile.ZipFile(zipname,"r") as zf:
        #Zipファイルに含まれるファイルを開く。今回のZIPは一つのテキストファイルのみ含む。
        for filename in zf.namelist():
            with zf.open(filename,"r") as f:
                #今回読むファイルはShift-JISなので指定してデコードする
                return f.read().decode("shift-jis")
    print("read_book ok")

#引数のテキストを分かち書きして配列にする ---(*4)
def split_words(text):
    node = mecab.parseToNode(text)
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if  hinshi in ["名詞"]:
            wakati_words.append(node.surface)
        elif hinshi in ["動詞", "形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words
    print("book_list ok")

#作品リストをDoc2Vecが読めるTaggedDocument形式にし、配列に追加する --- (*5)
documents = []
#作品リストをループで回す
for auther, book in book_list():
    #作品の文字列を取得
    words = read_book(auther, book)
    #作品の文字列を分かち書きに
    wakati_words = split_words(words)
    #TaggedDocumentの作成　文書=分かち書きにした作品　タグ=作者:作品名
    document = TaggedDocument(
        wakati_words, [auther["name"] + ":" + book["name"]])
    documents.append(document)

print("start leaning")
    
#TaggedDocumentの配列を使ってDoc2Vecの学習モデルを作成 --- (*6)
model = models.Doc2Vec(
    documents, dm=1, vector_size=300, window=5, min_count=1)

print("end leaning")

#Doc2Vecの学習モデルを保存
model.save('aozora.model')

print("モデル作成完了")

SyntaxError: ignored